<a href="https://colab.research.google.com/github/sahibpreetsingh12/100daysofcode/blob/master/dhoni_kohli_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -q keras
!mkdir test

In [31]:
from google.colab import drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
cp -r /content/gdrive/My\ Drive/train /content/

In [0]:
from glob import glob
from sklearn.model_selection import train_test_split

virat = glob('train/virat/*.jpeg')
dhoni = glob('train/dhoni/*.jpeg')

virat_train, virat_test = train_test_split(virat, test_size=0.30)
dhoni_train, dhoni_test = train_test_split(dhoni, test_size=0.30)

TRAIN_DIR = 'train'
TEST_DIR = 'test'

!mkdir test

!mkdir test/dhoni
files = ' '.join(dhoni_test)
!mv -t test/dhoni $files

!mkdir test/virat

files = ' '.join(virat_test)
!mv -t test/virat $files

In [0]:
from keras.models import Model
from keras.layers import Dense,GlobalAveragePooling2D,Dropout
from keras.applications.inception_v3 import InceptionV3,preprocess_input



In [0]:
classes=2
model=InceptionV3(weights='imagenet',include_top=False)
#to confirm we have got weights file see a file with .h5 extension

In [58]:
x = model.output#it is a tensor
print(x)

Tensor("mixed10_2/concat:0", shape=(?, ?, ?, 2048), dtype=float32)


In [0]:

x = GlobalAveragePooling2D(name='avgpool')(x)
#print(type(x),x)
x = Dropout(0.4)(x)
predictions = Dense(classes, activation='softmax')(x)
#print("sahib")
#print(predictions,type(predictions))
model = Model(inputs=model.input, outputs=predictions)
   
# transfer learning
for layer in model.layers:
    layer.trainable = False
      
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# **Image Augmentation**

Keras provides the ImageDataGenerator class that defines the configuration for image data preparation and augmentation. This includes capabilities such as:

Sample-wise standardization.
Feature-wise standardization.
ZCA whitening.
Random rotation, shifts, shear and flips.
Dimension reordering.
Save augmented images to disk.

In [60]:
from keras.preprocessing.image import ImageDataGenerator

WIDTH = 299
HEIGHT = 299
BATCH_SIZE = 32

# data prep
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

validation_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,#TRAIN DIR is train
    target_size=(HEIGHT, WIDTH),
		batch_size=BATCH_SIZE,
		class_mode='categorical')
    
validation_generator = validation_datagen.flow_from_directory(
    TEST_DIR,#TEST DIR is test
    target_size=(HEIGHT, WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical')



Found 19 images belonging to 2 classes.
Found 8 images belonging to 2 classes.


In [61]:
EPOCHS = 5
BATCH_SIZE = 4
STEPS_PER_EPOCH = 4
VALIDATION_STEPS = 4

MODEL_FILE = 'cricketer.model'

history = model.fit_generator(
    train_generator,
    epochs=EPOCHS,
    steps_per_epoch=STEPS_PER_EPOCH,
    validation_data=validation_generator,
    validation_steps=VALIDATION_STEPS)
  
model.save(MODEL_FILE)

Epoch 1/5
4/4 [==============================] - 41s 10s/step - loss: 0.7037 - acc: 0.6184 - val_loss: 0.5790 - val_acc: 0.6875
Epoch 2/5
4/4 [==============================] - 30s 7s/step - loss: 0.7937 - acc: 0.4737 - val_loss: 0.6218 - val_acc: 0.6562
Epoch 3/5
4/4 [==============================] - 31s 8s/step - loss: 0.7936 - acc: 0.5132 - val_loss: 0.5405 - val_acc: 0.7812
Epoch 4/5
4/4 [==============================] - 30s 8s/step - loss: 0.7340 - acc: 0.5789 - val_loss: 0.7147 - val_acc: 0.5312
Epoch 5/5
4/4 [==============================] - 31s 8s/step - loss: 0.7478 - acc: 0.5263 - val_loss: 0.6481 - val_acc: 0.5938


In [0]:
import numpy as np
from keras.preprocessing import image
from keras.models import load_model
from PIL import Image

In [66]:

def predict(model, img):
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    preds = model.predict(x)
    print(preds)
    return preds[0]

img = image.load_img('vk14.jpeg', target_size=(HEIGHT, WIDTH))
preds1 = predict(load_model(MODEL_FILE), img)



[[0.5764661  0.42353392]]


In [67]:
generator= train_datagen.flow_from_directory("train", batch_size=32)
label_map = (generator.class_indices)
print(label_map)
#y_classes = preds.argmax(axis=-1)
#print(y_classes)
y_classes = preds1.argmax(axis=-1)
print(y_classes)

Found 19 images belonging to 2 classes.
{'dhoni': 0, 'virat': 1}
0
